In [2]:
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import KFold



In [3]:
data_raw = np.load(r"..\data\raw\fashion_train.npy")
data_img = [np.reshape(image[:784],(28,28)) for image in data_raw]

Below is the template matching with the averaged out templates for each category.
Results are surprisingly good, given that the model is pretty straightforward in the approach

In [4]:
def get_scores_main(mean_conf_matrix):
    TP = np.diag(mean_conf_matrix)
    FP = np.sum(mean_conf_matrix,axis=0)-TP
    FN = np.sum(mean_conf_matrix,axis=1)-TP
    accuracy = sum(TP)/sum(TP+FP)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F1 = 2*((precision*recall)/(precision+recall))
    return [accuracy, precision, recall, F1]
    
# get_scores(mean_conf_matrix)

In [18]:
def get_col_name(row):    
    b = (results_df.loc[row.name] == row['Template_score'])
    return b.index[b.argmax()]


kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(data_raw)
collect = []
all_scores = []
score_frame = pd.DataFrame([[data_raw[i,-1]]+[None]*5 for i in range(len(data_raw))])
for i,(train_index, test_index) in enumerate(kf.split(data_raw)):
    
    print(f"Fold {i}")
    
    clothes = data_raw[train_index]
    test_clothes = data_raw[test_index]
    mean_clothes = [np.mean(clothes[clothes[:,-1]==x],axis=0) for x in range(5)]

    test_results = pd.DataFrame(test_index)
    results_df = pd.DataFrame()

    for i in range(len(mean_clothes)-1,-1,-1):
        results = np.sqrt(np.sum(np.square((test_clothes-mean_clothes[i])),axis=1))
        results_df.insert(0,i,pd.Series(results).values,allow_duplicates=True)
        score_frame.iloc[test_index,i+1] = pd.Series(results).values

    test_results['Actual_score'] = test_clothes[:,-1]
    test_results.insert(2,"Template_score",results_df.min(axis = 1))
    test_results['Template_score'] = test_results.apply(get_col_name, axis=1)
                 
    y_test = test_results['Actual_score']
    y_pred = test_results['Template_score']
    # print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
    # print('Classification Report:\n',classification_report(y_test, y_pred))
    fold_scores = get_scores_main(confusion_matrix(y_test, y_pred))
    # fold_scores +=
    all_scores.append(fold_scores)
    collect.append(confusion_matrix(y_test, y_pred))


Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [ ]:
# roc_auc_score(y_test,y_pred, multi_class="ovo")
# score_frame

,0,1,2,3,4,5
0,0,786.163332,1893.617989,1656.606032,1508.050258,1265.243842
1,1,2036.162052,1216.147297,2521.655699,1644.656337,2072.8969
2,0,948.560884,2171.532519,1861.113602,1777.445263,1569.230653
3,3,1522.75006,1475.920741,2057.235609,1309.609253,1591.616548
4,4,1548.300035,2092.940959,1342.575013,1792.842291,1041.100386
...,...,...,...,...,...,...
9995,4,1552.679681,2298.016194,2018.83842,1724.470566,1572.100662
9996,0,1241.296035,2140.469132,1737.109584,1698.732479,1393.658827
9997,3,2107.370679,1706.699109,2663.519216,1362.375932,2288.358855
9998,0,874.3799,1822.800565,1801.221329,1379.822873,1360.70871


In [21]:
score_frame.to_csv(r"..\data\interim\template_matching.csv")

In [ ]:
mean_conf_matrix = np.mean(collect,axis=0)
var_conf_matrix = np.var(collect,axis=0)
print("Mean confusion matrix\n", mean_conf_matrix)
print()
print("Variance of mean confusion matrix\n", var_conf_matrix)


Mean confusion matrix
 [[295.4  17.4  10.   54.4  29.4]
 [  9.4 349.8   4.6  18.8   6.8]
 [  2.2   4.  260.2   4.4 129.4]
 [ 23.8  11.4   4.8 334.4  26.6]
 [ 80.4   8.6 126.8  30.2 156.8]]

Variance of mean confusion matrix
 [[273.84   0.64   2.8   58.64  13.04]
 [  9.84 169.36   2.64  14.96   7.76]
 [  1.76   4.4  151.76   7.44  97.84]
 [ 52.56  12.64   2.56  41.84  13.84]
 [139.44   3.44 106.16  31.36 186.96]]


In [ ]:

print("Accuracy:", round(np.mean([all_scores[i][0] for i in range(5)],axis=0),4))
print("Std Accuracy:", round(np.std([all_scores[i][0] for i in range(5)]),4))
print()
print("Precision:",list(map(lambda x:round(x,4),np.mean([all_scores[i][1] for i in range(5)],axis=0))))
print("Std Precision:",list(map(lambda x:round(x,4),np.std([all_scores[i][1] for i in range(5)],axis=0))))
print()
print("Recall:",list(map(lambda x:round(x,4),np.mean([all_scores[i][2] for i in range(5)],axis=0))))
print("Std Recall:",list(map(lambda x:round(x,4),np.std([all_scores[i][2] for i in range(5)],axis=0))))
print()
print("F1:",list(map(lambda x:round(x,4),np.mean([all_scores[i][3] for i in range(5)],axis=0))))
print("Std F1:",list(map(lambda x:round(x,4),np.std([all_scores[i][3] for i in range(5)],axis=0))))


Accuracy: 0.6983
Std Accuracy: 0.0146

Precision: [0.7184, 0.8941, 0.6404, 0.7565, 0.4489]
Std Precision: [0.0313, 0.0051, 0.0257, 0.0122, 0.0283]

Recall: [0.7265, 0.8984, 0.6506, 0.8344, 0.3892]
Std Recall: [0.0061, 0.0082, 0.0236, 0.0226, 0.0198]

F1: [0.7221, 0.8962, 0.6454, 0.7933, 0.4167]
Std F1: [0.0175, 0.004, 0.0242, 0.0108, 0.0214]


Below is the template matching by taking the mean and dividing it over the variance, measuring the distance from that. Either I have done it in a not intended way or it is just not a reliable way of predicting values

In [ ]:
def get_col_name(row):    
    b = (results_df.loc[row.name] == row['Template_score'])
    return b.index[b.argmax()]


kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(data_raw)
collect = []

for i,(train_index, test_index) in enumerate(kf.split(data_raw)):
    
    print(f"Fold {i}:")
    
    clothes = data_raw[train_index]
    test_clothes = data_raw[test_index]
    mean_clothes = [np.mean(clothes[clothes[:,-1]==x])/np.var(clothes[clothes[:,-1]==x]) for x in range(5)]

    test_results = pd.DataFrame(test_index)
    results_df = pd.DataFrame()

    for i in range(len(mean_clothes)-1,-1,-1):
        results = abs(np.mean(test_clothes,axis=1)/np.var(test_clothes,axis=1)-mean_clothes[i])
        results_df.insert(0,i,pd.Series(results).values,allow_duplicates=True)
    
    test_results['Actual_score'] = test_clothes[:,-1]
    test_results.insert(2,"Template_score",results_df.min(axis = 1))
    test_results['Template_score'] = test_results.apply(get_col_name, axis=1)
                 
    y_test = test_results['Actual_score']
    y_pred = test_results['Template_score']
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
    print('Classification Report:\n',classification_report(y_test, y_pred))
    
    collect.append(confusion_matrix(y_test, y_pred))
allones = np.array(collect).sum(axis=0)



Fold 0:
Confusion Matrix:
 [[ 89   8 229  36  29]
 [ 52 227  18  91   9]
 [ 66   1 284  12  37]
 [ 89 126  74 101  22]
 [ 55   2 308  14  21]]
Classification Report:
               precision    recall  f1-score   support

           0       0.25      0.23      0.24       391
           1       0.62      0.57      0.60       397
           2       0.31      0.71      0.43       400
           3       0.40      0.25      0.30       412
           4       0.18      0.05      0.08       400

    accuracy                           0.36      2000
   macro avg       0.35      0.36      0.33      2000
weighted avg       0.35      0.36      0.33      2000

Fold 1:
Confusion Matrix:
 [[ 89  14 238  35  33]
 [ 46 244  17 100   7]
 [ 77   0 285  11  16]
 [ 71 126  71 115  16]
 [ 52   4 302  11  20]]
Classification Report:
               precision    recall  f1-score   support

           0       0.27      0.22      0.24       409
           1       0.63      0.59      0.61       414
           2  